<h1 align="center">TBR Analysis (Water - Graphene Oxide)</h1>


**Author:** F. Tarulli, Politecnico di Torino, Italy - Jun 7, 2025

**Co-Author** P. De Angelis, Politecnico di Torino, Italy

<br>

Script loads transient LAMMPS outputs, computes ∫ΔT*dt, finds linear regime,  
fits OLS to extract Kapitza resistance ($\mathrm{R_K}$) and conductance ($\mathrm{G_K}$),  
and plots results. <br>It processes a single simulation; to achieve robust estimates of the TBR, the analysis should be repeated over an ensemble of independent runs.

**Adjust “Variables” section before running:**

- `TBulk`  
  *If True, reads water bulk temperature; if False, uses slab-based temperature profile*  
- `h_slab`  
  *Slab half-thickness in Å for averaging water temperature around the graphene*  
- `oxid`  
  *Oxidation degree (%) for the –OH coverage on the graphene sheet*  


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.integrate import cumulative_trapezoid
import os
from scipy.constants import N_A
from tqdm.auto import tqdm
from scipy.optimize import minimize_scalar
import statsmodels.api as sm
from time import time

In [ ]:
def read_Tgraph_chunks(base_remote_path, file_name, k):
    Temp_chunks = []
    with open(f'{base_remote_path}/slab_position_check.txt', 'r') as file:
        for _ in range(3):
            next(file)
        line = next(file).strip().split()
        Nchunks_tot = int(line[1])
    with open(f'{base_remote_path}/{file_name}', 'r') as file:
        for i in range(k):
            if i == 0:
                # Skip the first 4 lines for the first chunk
                for _ in range(4):
                    next(file)
            else:
                # Skip the first 2 lines for subsequent chunks
                next(file)
            
            # Read the next N lines
            chunk = []
            for _ in range(Nchunks_tot):
                line = next(file).strip()
                values = list(map(float, line.split()))
                chunk.append(values)
            
            Temp_chunks.append(np.array(chunk))
    return Temp_chunks, Nchunks_tot

def process_chunks(Temp_chunks, Nchunks_tot, Nchunks, slab_skip=3): 
    k = len(Temp_chunks)
    T_slab_positive = np.zeros(k)
    T_slab_negative = np.zeros(k)
    T_bulk_chunk = np.zeros(k)

    for i in range(k):
        up = Nchunks_tot // 2 + int(slab_skip) 
        down = Nchunks_tot // 2 - 1 - int(slab_skip) 
        
        # Average layers above graphene
        positive_layers = [Temp_chunks[i][up + j, 1] for j in range(Nchunks-slab_skip)]
        T_slab_positive[i] = np.mean(positive_layers)
        
        # Average layers below graphene
        negative_layers = [Temp_chunks[i][down - j, 1] for j in range(Nchunks-slab_skip)]
        T_slab_negative[i] = np.mean(negative_layers)

        # Average temperature across the entire chunk
        T_bulk_chunk[i] = np.mean(Temp_chunks[i][:, 1])
    
    # Mean temperature of both slabs
    T_slab_mean = np.mean(np.vstack((T_slab_positive, T_slab_negative)), axis=0)
    return T_slab_positive, T_slab_negative, T_bulk_chunk, T_slab_mean


def find_breakpoint_iterative(x, y, min_points=10, maxiter=100, penalty_npoint=0.1, penalty_const=0.1):
    def mse_break(x0):
        idx = np.searchsorted(x, x0, 'right')
        c0 = np.mean(y[:10])
        if idx < min_points:
            idx = min_points
            return np.float64(1e100)
        xi, yi = x[:idx], y[:idx]
        yl = y[idx:]
        m, c = np.polyfit(xi, yi, 1)
        const_par = m*x0 + c0
        return np.sum((yi - (m*xi + c0))**2) + penalty_const*np.sum((yl - const_par)**2)  + 0*np.abs(np.log(len(x)/len(xi)))*penalty_npoint

    res = minimize_scalar(
        mse_break,
        bounds=(x[min_points], x[-1]),
        method='bounded',
        options={'maxiter': maxiter}
    )
    return res.x

# Variables

In [ ]:
TBulk = False    # If True, reads water bulk temperature; if False, uses slab-based temperature profile
h_slab = 9       # Slab half-thickness in Å for averaging water temperature around the graphene
oxid = 50        # Oxidation degree (%)

base_remote_path = Path('../../lammps/TBR/transient')

# Load

In [ ]:
DATA_ALL = {}

In [ ]:
temps_w = []
temps_g = []
pe_g = []
ke_g = []
etot = []
print(f" OXIDATION DEGREE = {oxid:3d}% ".center(100, "="))

try:
    file_liquid = 'system_h2o.txt'
    file_nanoparticles = 'system_graph.txt'
    file_chunk = 'temp_chunk_bias_1A.out'
    file_Pot = 'potential_graph.txt'
    file_Kin = 'kinetic_graph.txt'
    temps_g.append(pd.read_csv(os.path.join(base_remote_path, file_nanoparticles), sep=r'\s+',  skiprows=2, names=["step", f"temp"]))
    if TBulk:
        temps_w.append(pd.read_csv(os.path.join(base_remote_path, file_liquid), sep=r'\s+', skiprows=2, names=["step", f"temp"]))
    else: 
        Tchunks, tot_chunks = read_Tgraph_chunks(base_remote_path, file_chunk, temps_g[0].shape[0])
        T_slab_positive, T_slab_negative, T_bulk_chunk, T_slab_mean = process_chunks(Tchunks,tot_chunks,h_slab)
        df_slab = pd.DataFrame({"step": temps_g[-1]["step"].values, f"temp": T_slab_mean })
        temps_w.append(df_slab)
    pe_g.append(pd.read_csv(os.path.join(base_remote_path, file_Pot), sep=r'\s+', skiprows=2, names=["step", "pe"]))
    ke_g.append(pd.read_csv(os.path.join(base_remote_path, file_Kin), sep=r'\s+', skiprows=2, names=["step", "ke"]))  
    etot.append( pd.DataFrame(data={"step": pe_g[-1]["step"], "etot": pe_g[-1]["pe"] + ke_g[-1]["ke"]}) )
except Exception:
    print("Error, files not found\n")

rep_Tgr = pd.concat(temps_g, axis=1)
rep_etot = pd.concat(etot, axis=1)

rep_Tw = pd.concat(temps_w, axis=1)

DATA_ALL[oxid] = {
    "Tw": rep_Tw.copy(),
    "Tgr": rep_Tgr.copy(),
    "etot": rep_etot.copy(),
}

# Concatenate

In [ ]:
CONCAT_ALL_DATA = {}
for oxid, data in tqdm(DATA_ALL.items(),desc="Concatenate data", total=len(DATA_ALL)):
   
    CONCAT_ALL_DATA[oxid] = {
        "time_s": data["Tw"].iloc[:,0].values * 1e-15, 
        "Tw": np.column_stack([data["Tw"][f"temp"].values])[5000:],
        "Tgr": np.column_stack([data["Tgr"][f"temp"].values])[5000:],
        "etot": np.column_stack([data["etot"][f"etot"].values])[5000:]    
    }
    Nt = CONCAT_ALL_DATA[oxid]["time_s"].shape[0]
    int_t_rep = np.zeros((Nt, 1))

    dt_i = data["Tgr"].loc[:, f"temp"].values - data["Tw"].loc[:, f"temp"].values
    int_t_rep[1:, 0] = cumulative_trapezoid(dt_i, CONCAT_ALL_DATA[oxid]["time_s"])

    CONCAT_ALL_DATA[oxid]['integral'] = int_t_rep[5000:] - int_t_rep[5000]

# $R_K$

In [ ]:
SLOPE_ALL = {}
AREA = {}
try:
    with open(f'{base_remote_path}/water-graph_transient.dump', 'r') as file:
        for _ in range(5):
            next(file)
        x_line = next(file).strip().split()
        y_line = next(file).strip().split()
    xx = float(x_line[1]) - float(x_line[0]) 
    yy = float(y_line[1]) - float(y_line[0])
    AREA[oxid] = 2 * xx*1e-10 * yy*1e-10
except:
    print("ERROR: trajectory file not found.\nImport AREA variable manually.\n")

In [ ]:
for oxid, data in CONCAT_ALL_DATA.items():
    start_s = time()
    print(f" OXIDATION DEGREE: {oxid:3d}% ".center(120, "="))
    diffT = data['Tgr'] - data['Tw']
    integral = data['integral']
    etot = data['etot']
    scaleE = etot.std()
    scaleInt = integral.std()
    t0_min = find_breakpoint_iterative(integral[:,0]/scaleInt, etot[:,0]/scaleE, min_points=9, maxiter=500, penalty_npoint=2, penalty_const=1) * scaleInt
    x_fit = integral[integral<t0_min]
    y_fit = etot[integral<t0_min]
    x_fit = sm.add_constant(x_fit) 
    robust_model = sm.OLS(y_fit, x_fit)
    results = robust_model.fit()
    slope = results.params[1]
    intercept = results.params[0]
    x_fit = integral[integral<t0_min]
    y_fit = etot[integral<t0_min]
    x_fit = sm.add_constant(x_fit)  
    robust_model = sm.OLS(y_fit, x_fit)
    results_rep = robust_model.fit()

    x_pred = np.linspace(integral.min(), t0_min, 100)
    y_med = intercept + slope * x_pred

    SLOPE_ALL[oxid] = {
        "Rk_m" : - AREA[oxid]/ (slope* 4184 / N_A),
    }

    SLOPE_ALL[oxid]["Gk_m"] = 1/SLOPE_ALL[oxid]["Rk_m"]

    print(f"Kapitza resistance = {SLOPE_ALL[oxid]["Rk_m"]:1.2e} (K*m^2/W)")
    print(f"Kapitza conductance = {SLOPE_ALL[oxid]["Gk_m"]:1.2e} (W/K/m^2)")
    
elapsed_time = time() - start_s
print(f"\n(Elapsed time {elapsed_time:.2f} s)")

# Plot

In [ ]:
indx0 = integral[:, 0] < t0_min
pred = results_rep.get_prediction(sm.add_constant(x_pred))
y_pred = pred.predicted

plt.scatter(integral[indx0, 0]*1e9, etot[indx0, 0]*1e-3,
            s=4, alpha=0.6,
            color="#1f77b4",
            label="Raw data")

plt.plot(x_pred*1e9, y_pred*1e-3,
         color="#ff7f0e",
         lw=2, ls="--",
         label="Linear fit")

plt.xlabel(r"$\int \Delta T\,dt\;(\mathrm{K\cdot ns})$", fontsize=12)
plt.ylabel("Energy (Mcal/mol)", fontsize=12)
plt.title(f"TBR (Graphene Oxide {oxid}%)", fontsize=14)
plt.legend(frameon=True, loc="best")
plt.grid(True, linestyle=":", alpha=0.5)

plt.tight_layout()
plt.show()